In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as m
import cantera as ct

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.data import H2_combustion2
from src.edmd import TruncEDMD
from src.kernels import RBFKernel
from src.training_edmd import GD_edmd_training
from src.visualizations import legend_without_duplicate_labels, dynamicscombust,gridsearchnorm,traininglandscapenorm,OOS_system_normalize
from matplotlib.legend_handler import HandlerTuple

In [ ]:
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

Define intiial conditions and time interval then plot the result of the original system

In [ ]:
dynamic_system="non_steady_state_H2_single"
f, ax = plt.subplots(1, 3, figsize=(10, 3.75))
f1, ax1 = plt.subplots(1, 3, figsize=(10, 3.75))
timesteps=200#133
factor=4
t_end=0.25#0.0006
t_begin=0.11

initial_conditions= np.array([[0.06,776],[0.06,778],[0.06,780],[0.06,782],[0.06,784],[0.06,786],[0.06,788],[0.06,790],[0.06,792]])

x,y,initial_conditions_norm,original_norm,original,t_eval_frac=dynamicscombust(H2_combustion2,initial_conditions,t_end,timesteps,factor,t_begin)
timesteps=len(t_eval_frac)


for ic, condition in enumerate(initial_conditions_norm):
    
    ax[0].plot(original_norm[ic*(timesteps):ic*(timesteps)+(timesteps), 0],original_norm[ic*(timesteps):ic*(timesteps)+(timesteps), 1],)
    
    ax[1].plot(t_eval_frac,original_norm[ic*(timesteps):ic*(timesteps)+(timesteps), 0])
    ax[2].plot(t_eval_frac,original_norm[ic*(timesteps):ic*(timesteps)+(timesteps), 1])
    ax1[0].plot(original[ic*(timesteps):ic*(timesteps)+(timesteps), 0],original[ic*(timesteps):ic*(timesteps)+(timesteps), 1],)
    ax1[1].plot(t_eval_frac,original[ic*(timesteps):ic*(timesteps)+(timesteps), 0])
    ax1[2].plot(t_eval_frac,original[ic*(timesteps):ic*(timesteps)+(timesteps), 1])



ax[0].grid()
ax[0].set_xlabel("H2 Mole Fraction")
ax[0].set_ylabel("Temperature")


ax[1].grid()
ax[1].set_xlabel("Time (sec)")
ax[1].set_ylabel("H2 Mole Fraction")

ax[2].grid()
ax[2].set_xlabel("Time (sec)")
ax[2].set_ylabel("Temperature")

ax1[0].grid()
ax1[0].set_xlabel("H2 Mole Fraction")
ax1[0].set_ylabel("Temperature(K)")
#ax[0].set_ylim(775,790)

ax1[1].grid()
ax1[1].set_xlabel("Time (sec)")
ax1[1].set_ylabel("H2 Mole Fraction")

ax1[2].grid()
ax1[2].set_xlabel("Time (sec)")
ax1[2].set_ylabel("Temperature(K)")



f.tight_layout()
f1.tight_layout()

Perform Grid Search

In [ ]:

initial_conditions= np.array([[0.06,776],[0.06,778],[0.06,780],[0.06,782],[0.06,784],[0.06,786],[0.06,788],[0.06,790],[0.06,792]])
initial_conditions_sample=np.array([[0.06,782.5] , [0.06,783.2] , [0.06,786]])
method = TruncEDMD()
parameters = np.array([0.1,0.5,1.0,1.5,2.0])#np.array([1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0])

truncs=np.array([5,10,15,20,25,30])#np.array([8,9,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,52,54,56,58])

factors=np.array([4])

t_begin=0.11

t_end=0.25
timesteps=160
kernel = RBFKernel
index=0

lossperstep,OOSlossperstep,optim=gridsearchnorm(factors,truncs,parameters,initial_conditions,
                                                initial_conditions_sample,timesteps,t_end,method,
                                                kernel,H2_combustion2,t_begin)

Plot Results of Grid Search

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))  # 1 row, 2 columns of subplots

im1 = ax[0].imshow(np.log(OOSlossperstep),extent=[5,30,0.1,2.0],aspect=11.5,origin="lower")
cbar1 = fig.colorbar(im1, ax=ax[0],shrink=0.6,)  # Colorbar for the first image
cbar1.set_label("Log(Error)")
ax[0].set_xlabel("Number of Eigenfunctions")
ax[0].set_ylabel("Free Parameter θ")



ax[0].set_title("Average Out-of-Sample Trajectory Error")  # Optionally set a title for the first subplot


im2 = ax[1].imshow(np.log(lossperstep),extent=[5,30,0.1,2.0],aspect=11.5,origin="lower")
cbar2 = fig.colorbar(im2, ax=ax[1],shrink=0.6)  # Colorbar for the second image

ax[1].set_xlabel("Number of Eigenfunctions")
ax[1].set_ylabel("Free Parameter θ")
ax[1].set_title("In-sample Loss")  # Optionally set a title for the second subplot

cbar2.set_label("Log(Loss)")

plt.tight_layout()

min=np.where(optim == np.nanmin(optim))
print(min)
# Show the plot
plt.show()


Explore the training landscape

In [ ]:
#x_next_DMD=edmd.trajectory_visualization_DMD(x,y,LinearKernel())
initial_conditions =np.array([[0.08,775],[0.08,778.5],[0.08,782]])
t_end=0.25#0.0006
timesteps=160
trunc=15
method = TruncEDMD()
factor=4
parameters = np.linspace(0.05, 50, num=200)
func=H2_combustion2
t_begin=0.11
kernel= RBFKernel

losstrain,min_loss=traininglandscapenorm(initial_conditions,t_end,timesteps,factor,trunc,method,
                          kernel,parameters,func,t_begin)


Plot the Training Landscape

In [ ]:
f, ax = plt.subplots()
ax.plot( parameters, np.log(losstrain), label='Training Loss')

ax.set_xlabel('Free Parameter θ')
ax.set_ylabel('Log(In-Sample Loss)')
ax.set_title("Loss Landscape") 

f.tight_layout()


Perform Training using optimized paramters

In [ ]:
method = TruncEDMD()
parameter = torch.tensor(1.0, requires_grad=True)
kernel = RBFKernel([parameter])
initial_conditions= np.array([[0.08,775],[0.08,778.5],[0.08,782]])
t_end=0.25#0.0006
timesteps=170
trunc=15
factor=4
t_begin=0.11
func= H2_combustion2
num_epochs=10


x,y,initial_conditions_norm,original_norm,original,t_eval_frac=dynamicscombust(func,initial_conditions,
                                                                                t_end,timesteps,factor,t_begin)

train_loss, min_param = GD_edmd_training(num_epochs,1, x.T, y.T, kernel, method,trunc, 
     lr=1e-1,full=True, penalty=True)

fig, ax = plt.subplots()
plt.plot( np.array(range(0,num_epochs)), np.array(train_loss), label='Training Loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.legend()
plt.show()
print(list(kernel.parameters()))



Visualize Training

In [ ]:
fig, ax = plt.subplots()
plt.plot( np.array(range(0,num_epochs)), np.array(np.log(train_loss[0:])), label='Training Loss')

ax.set_ylabel('Log(In-Sample Loss)')
ax.set_title("Training Landscape") 

plt.show()
print(list(kernel.parameters()))


Plot and report the in-sample and out of sample loss for the best paramter combination

In [ ]:

initial_conditions= np.array([[0.06,776],[0.06,778],[0.06,780],[0.06,782],[0.06,784],[0.06,786],[0.06,788],[0.06,790],[0.06,792]])

t_end=0.25#0.0006
timesteps=160
func= H2_combustion2
dt=t_end/timesteps
t_eval=np.linspace(0, t_end, timesteps)
trunc=42
factor=4
method = TruncEDMD()
t_begin=0.11
#parameter = torch.tensor(0.4, requires_grad=True)
parameter=torch.tensor(2.0,requires_grad=True) #33.2791   
kernel = RBFKernel([parameter])

x,y,initial_conditions_norm,original_norm,original,t_eval_frac=dynamicscombust(func,
                                                    initial_conditions,t_end,timesteps,factor,t_begin)
Kernel_results=method.edmd_computations(x.T,y.T,kernel,trunc)
method.preloss_computation_full()
loss = method.loss_full(kernel, x.T, y.T, penalty=True)
f, ax = plt.subplots(1, 3, figsize=(12, 4))

for ic,condition in enumerate(initial_conditions_norm):

    condition=np.array([condition],ndmin=2)
    Fx=method.single_tradjectory(t_eval_frac,condition.T,x.T,y.T,kernel,Kernel_results) 
   
    ax[0].plot(Fx[:,0],Fx[:,1])
    ax[0].set_xlabel("H2 Mole Fraction")
    ax[0].set_ylabel("Temperature ")
    ax[0].grid()
    ax[1].plot(t_eval_frac,Fx[:,0],)
    ax[1].set_xlabel("Time (sec)")
    ax[1].set_ylabel("H2 Mole Fraction")
    ax[1].grid()
    ax[2].plot(t_eval_frac,Fx[:,1],)
    ax[2].set_xlabel("Time (sec)")
    ax[2].set_ylabel("Temperature ")
    ax[2].grid()
    
   
    f.tight_layout()
print("loss",loss)


In [ ]:
#out of sample specification#################################################################   
f, ax = plt.subplots(1, 3, figsize=(12, 4))


initial_conditions_sample=np.array([[0.06,782.5] , [0.06,783.2] , [0.06,786]])
line=np.empty(len(initial_conditions_sample))

x,y,initial_conditions_norm,original_norm,original,t_eval_frac=dynamicscombust(func,
                                                initial_conditions,t_end,timesteps,factor,t_begin)
Kernel_results=method.edmd_computations(x.T,y.T,kernel,trunc)
method.preloss_computation_full()

#timesteps_sample=25
#dt=t_end/len(t_eval_frac)
#t_end_sample =timesteps_sample*dt
t_eval_sample = t_eval_frac

sumloss=0
for ic,condition in enumerate(initial_conditions_sample):
     
        t_eval=np.linspace(0, t_end, timesteps) #normal trajectory here
        dynamics_norm,initial_condition_norm_sample=OOS_system_normalize(t_eval,condition,original,func)
        int_begin=int(t_begin//(t_end/timesteps)) #change here!!!!!
        dynamics_norm=dynamics_norm[int_begin::factor]

        initial_condition_norm_sample=np.array([initial_condition_norm_sample],ndmin=2)
        predicted=method.single_tradjectory(t_eval_frac,initial_condition_norm_sample.T,x.T,y.T,kernel,Kernel_results) 
        sumloss = sumloss+np.sqrt(np.sum(np.square((abs(predicted-dynamics_norm))))/len(t_eval_frac))
       
        ax[0].plot(predicted[:,0],predicted[:,1],0.1,c="red",linestyle='--', label="Predicted" )   
        ax[0].plot(dynamics_norm[:,0],dynamics_norm[:,1],0.1,c="black", label="Original" )  
        ax[0].scatter(initial_condition_norm_sample[:,0],initial_condition_norm_sample[:,1]) 
        ax[0].set_xlabel("H2 Mole Fraction")
        ax[0].set_ylabel("Temperature ")
        ax[1].set_xlim([0.1,0.252])
        ax[2].set_xlim([0.1,0.252])
        ax[0].grid()
        ax[1].plot(t_eval_sample,predicted[:,0],0.1,c="red",linestyle='--', label="Predicted" )
        ax[2].plot(t_eval_sample,predicted[:,1],0.1,c="red",linestyle='--', label="Predicted" )
        ax[1].set_xlabel("Time (sec)")
        ax[1].set_ylabel("H2 Mole Fraction")
        ax[1].grid()
        ax[2].set_xlabel("Time (sec)")
        ax[2].set_ylabel("Temperature ")
        ax[2].grid()
      
        ax[1].plot(t_eval_sample,dynamics_norm[:,0],0.1,c="black", label="Original" )  
        ax[2].plot(t_eval_sample,dynamics_norm[:,1],0.1,c="black", label="Original" ) 
        legend_without_duplicate_labels(ax[0])
l2=sumloss/len(initial_conditions_sample)          

f.tight_layout()
print("L2_norm",l2)     
   

